## Jul AdventKalender D23

https://adventofcode.com/2022/day/23

In [1]:
import numpy as np
from itertools import cycle
from collections import defaultdict

#### Day 23.1 

Given a map of locations of elves(#) and empty tiles(.):

    ....#..
    ..###.#
    #...#.#
    .#...##
    #.###..
    ##.#.##
    .#..#..

For each round, there are two parts: considering where to move and actually moving.

First half round, each Elf considers the **8 positions** adjacent to themself. If **no other Elves** are in one of those eight positions, the Elf does **not** do anything during this round. Otherwise, the Elf looks in each of four directions in the following order and proposes moving one step in the first valid direction:

* If there is no Elf in the **N, NE, or NW** adjacent positions, the Elf proposes moving **north** one step.
* If there is no Elf in the **S, SE, or SW** adjacent positions, the Elf proposes moving **south** one step.
* If there is no Elf in the **W, NW, or SW** adjacent positions, the Elf proposes moving **west** one step.
* If there is no Elf in the **E, NE, or SE** adjacent positions, the Elf proposes moving **east** one step.

The second half round, each Elf moves to their proposed destination tile if they were the **only** Elf to propose moving to that position. If **two or more** Elves propose moving to the same position, **none** of those Elves move.

Finally, at the end of the round, the first direction the Elves considered is moved to the end of the list of directions.

Count the number of empty ground tiles contained by the smallest rectangle that contains every Elf.

In [2]:
def readFile(file_name):
    data_elves = {}
    f = open(file_name, "r")
    line_id = 0
    while True:
        line = f.readline()
        if not line:
            break
        row = line.strip()
        for i in range(len(row)):
            if row[i] == '#':
                data_elves[(line_id, i)] = True # dictionary to store positions, fast to search by position value
        line_id += 1
    f.close()
    return data_elves

In [3]:
# 8 directions
directions = [(-1,0),(1,0),(0,-1),(0,1), # 0 north, 1 south, 2 west, 3 east
              (-1,1),(1,1),(1,-1),(-1,-1)] # 4 NE, 5 SE, 6 SW, 7 NW
check_list = [(0,4,7),(1,5,6),(2,7,6),(3,4,5)] # priority of directions to check, correspond to index in directions

# N, NE, or NW => N
# S, SE, or SW => S
# W, NW, or SW => W
# E, NE, or SE => E

def roundFirstHalf(start_dir):
    global position_elves
    start_dir_id = next(start_dir)
    propose_elves = defaultdict(list)
    for pos_elf in position_elves:
        pos_elf_surrounds = [(pos_elf[0]+dire[0],pos_elf[1]+dire[1]) for dire in directions]
        # if all empty in surrounding positions
        check_res = np.zeros(8) #0:empty, 1:elf. stored for later use
        for s in range(8):
            if pos_elf_surrounds[s] in position_elves:
                check_res[s] = 1
        if sum(check_res) == 0:  # no move
            continue
        # purpose moving position if all checking pos are empty, according to priority (check_list)
        for c in range(4):
            check_ids = check_list[(c+start_dir_id)%4]
            if sum([check_res[cid] for cid in check_ids]) == 0:
                pos_proposed = (pos_elf[0]+directions[check_ids[0]][0],pos_elf[1]+directions[check_ids[0]][1])
                if pos_proposed in propose_elves:
                    propose_elves[pos_proposed].append(pos_elf)
                else:
                    propose_elves[pos_proposed] = [pos_elf]
                break
    return propose_elves

def roundSecondHalf(propose_elves):
    global position_elves
    for key, value in propose_elves.items():
        if len(value) == 1: # only one elf proposed this position to move
            position_elves[key] = True # elf moves to the porposed position
            del position_elves[value[0]] # from its original position
    return len(propose_elves) == 0 # no one proposed to moove

def getNumEmptyTiles():
    global position_elves
    row_min,row_max = float('inf'),float('-inf')
    col_min,col_max = float('inf'),float('-inf')
    for pos in position_elves:
        row_min = min(row_min, pos[0])
        row_max = max(row_max, pos[0])
        col_min = min(col_min, pos[1])
        col_max = max(col_max, pos[1])
    #print(row_min,row_max, col_min,col_max)
    return (row_max-row_min+1)*(col_max-col_min+1)-len(position_elves)

In [4]:
position_elves = readFile('data/input23.txt')
start_dir = cycle(range(4))
for _ in range(10):
    propose_elves = roundFirstHalf(start_dir)
    roundSecondHalf(propose_elves)
getNumEmptyTiles()

4181

#### Day 23.2

What is the number of the first round where no Elf moves?

In [6]:
from time import perf_counter

position_elves = readFile('data/input23.txt')
start_dir = cycle(range(4))
n_round = 0
start = perf_counter() 
while True:
    n_round += 1
    #print(n_round)
    propose_elves = roundFirstHalf(start_dir)
    if roundSecondHalf(propose_elves):
        break
end = perf_counter()
print('Total rounds:', n_round)
print('Completed in:', round((end - start) * 1000, 2), 'ms')

Total rounds: 973
Completed in: 18020.41 ms
